In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

## Web scraping for  job listings from the CareerJet website from the locations USA and Canada

In [ ]:
start_page = 1
max_pages = 100
location = "USA"  #Canada

list_to_df = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}

session = requests.Session()
for page in range(max_pages):

# https://www.careerjet.ca/jobs?s=&l=         ---> Canda
  url = f"https://www.careerjet.com/jobs/{location}?p={page}"
  try:
      response = session.get(url, headers=headers)
      response.raise_for_status()

      soup = BeautifulSoup(response.text, 'html.parser')
      job_tags = soup.find_all('article')

      for job in job_tags:
          title = job.find('a', title=True)['title'] if job.find('a',title=True) else "None"
          company = job.find('p', class_='company').get_text(strip=True) if job.find('p', class_='company') else "None"
          link = job.find('a', href=True)['href'] if job.find('a', href=True) else "None"
          link = link if link.startswith('http') else 'https://www.careerjet.com' + link

          list_to_df.append({
              "Title": title,
              "Company": company,
              "Link": link
          })

  except requests.exceptions.RequestException as e:
      print(f"Failed to retrieve HTML content for page {page}. Error: {e}")

df = pd.DataFrame(list_to_df)
df = df.drop_duplicates(subset=['Title', 'Link'])


df['Description'] = pd.Series(dtype='str')


for index, row in df.iterrows():
    job_link = row["Link"]

    try:
        response = session.get(job_link, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')


        description_tag = soup.find('section', class_="content")

        full_description = description_tag.get_text(strip=True) if description_tag else "None"
        df.at[index, 'Description'] = full_description

    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve description for {job_link}. Error: {e}")
        df.at[index, 'Description'] = "None"


df.to_csv('USA_data.csv', index=False)